# Investigate Price Correlations w/ Fundamentals

### For each metric of interest, X:

- create by-year metric data, X per share
- calculate the average ratio for price-to-X
- rate the price correlations using the following ways:
    - calculate the correlation between price and X
    - plot the price against the price-per-average-price-to-X-ratio

In [1]:
import json
import requests
import numpy as np
from datetime import datetime
from yahoo_fin import stock_info

## Determine the earliest dates of price and financials data to include in analysis

In [2]:
# set years interval for analysis
analysis_interval_years = 20

# get the earlist fiscal year accordingly
start_year = datetime.utcnow().year - analysis_interval_years
start_date_quote_history = datetime(start_year, 1, 1)
start_year_financials_history = start_year - 1

print("Start date of quote history data:", start_date_quote_history)
print("Start year of financials history data:", start_year_financials_history)

Start date of quote history data: 2001-01-01 00:00:00
Start year of financials history data: 2000


## Helper functions to pull and prep stock price data

(updated 10/26/2021)

- Switched from "yfinance" to "yahoo_fin" as the library to pull data from Yahoo Finance
- <font color=blue>Validation OK</font>:
    - Updated code tested on AAPL and the historical quotes now seem to be consistent with those on Yahoo Finance

In [3]:
def get_payload_json_quote_history_from_yfinance_df(df_quote_history, header='close'):
    """
    This helper function gets a cleaned-up payload json/dict from the dataframe of 
    historical quotes, which is returned by yfinance api (yfinance.Ticker('<symbol>').history)
    """
    
    _payload_json = {}
    _df = df_quote_history[header]
    
    # loop through all timestamps in the data:
    for timestamp in dict(_df):
        if not np.isnan(_df[timestamp]):
            _payload_json[timestamp.to_pydatetime()] = _df[timestamp]
            
    return _payload_json


def get_quote_history(symbol, start, end, interval='1mo', header='close'):
    """
    This helper function pulls raw historical quote data using the YFinance API, 
    and returns the cleaned up data in a dictionary of "<timestamp>: <price>".
    """
    
    # initialize an yfinance object, and get the historical quote data 
    # from the API
    df_quote_history = stock_info.get_data(symbol, start_date=start, end_date=end, 
                                           interval=interval)
    
    # return a "timestamp": "price" dictionary from the dataframe of historical quotes
    return get_payload_json_quote_history_from_yfinance_df(df_quote_history, 
                                                           header=header)


# testing

aapl_quote_history = get_quote_history('AAPL', 
                                       start=start_date_quote_history.strftime('%m/%d/%Y'),
                                       end=datetime.utcnow().strftime('%m/%d/%Y'))
aapl_quote_history

# testing successful

{datetime.datetime(2001, 1, 1, 0, 0): 0.3861609995365143,
 datetime.datetime(2001, 2, 1, 0, 0): 0.32589301466941833,
 datetime.datetime(2001, 3, 1, 0, 0): 0.39410701394081116,
 datetime.datetime(2001, 4, 1, 0, 0): 0.45517900586128235,
 datetime.datetime(2001, 5, 1, 0, 0): 0.35624998807907104,
 datetime.datetime(2001, 6, 1, 0, 0): 0.4151790142059326,
 datetime.datetime(2001, 7, 1, 0, 0): 0.33553600311279297,
 datetime.datetime(2001, 8, 1, 0, 0): 0.33125001192092896,
 datetime.datetime(2001, 9, 1, 0, 0): 0.276964008808136,
 datetime.datetime(2001, 10, 1, 0, 0): 0.3135710060596466,
 datetime.datetime(2001, 11, 1, 0, 0): 0.3803569972515106,
 datetime.datetime(2001, 12, 1, 0, 0): 0.39107099175453186,
 datetime.datetime(2002, 1, 1, 0, 0): 0.4414289891719818,
 datetime.datetime(2002, 2, 1, 0, 0): 0.38749998807907104,
 datetime.datetime(2002, 3, 1, 0, 0): 0.42267900705337524,
 datetime.datetime(2002, 4, 1, 0, 0): 0.4333930015563965,
 datetime.datetime(2002, 5, 1, 0, 0): 0.41607099771499634,
 d

## Helper functions to pull and prep stock fundamentals data

In [ ]:
def get_guru_data(symbol, data_type, api_token='b4bbdecd0f955e18a90fee818670dd94:42afdf0e68bee024983a72f8b6ad071d'):
    """
    This helper function pulls data from the GuruFocus API, for the given symbol
    and data type.
    """
    
    base_url = 'https://api.gurufocus.com/public/user/' + api_token + '/stock/'
    constructed_url = base_url + symbol + '/' + data_type
    
    r = requests.get(constructed_url)
    if r.status_code != 200:
        return "Error: the GuruFocus API service failed."
    else:
        return r.json()
    

# testing

aapl_financials_history = get_guru_data('AAPL', 'financials')
aapl_financials_history

# testing successful

In [ ]:
class FinancialMetric(object):
    """
    This class implements financial metrics as objects, along with 
    related operations.
    """
    
    def __init__(self, name, timestamps, start_year, values, numerify_values=True, drop_timestamps=['TTM']):
        """
        Constructor.
        
        Input:
            - "name": name of the metric.
            - "timestamps": a list (or list-like) of strings in the format of "%Y-%m"
            - "start_year": the earliest year in the input data to be used
            - "values": a list (or list-like) of values (could be strings) for the metric,
                        with each value corresponding to the timestamp value of the same position
                        in the input list of timestamps
            - "numerify_values": default=True; 
                                 if True attempt to convert all input values to float values
            - "drop_values": default=['TTM']; 
                             a list of strings to be removed from the input timestamps
                             list, and the associated value will also be removed from the input values 
                             list.
        """
        
        # if requested, ensure all values are converted to numeric values (float)
        if numerify_values:
            _values = [float(value) for value in values]
        else:
            _values = values

        # zip the two lists to form a dictionary
        _data = dict(zip(timestamps, _values))

        # form a new dictionary with pre-specified contraints on the "year" values
        _processed_data = {}
        for timestamp in _data:
            if timestamp in drop_timestamps:
                continue
            _timestamp_obj = datetime.strptime(timestamp, '%Y-%m')
            if _timestamp_obj.year >= start_year:
                _processed_data[_timestamp_obj] = _data[timestamp]
        
        # save the processed timestamps and values
        self.name = name
        self.timestamps = tuple(_processed_data.keys())
        self.values = tuple(_processed_data.values())
        
    def get_raw_data(self):
        """
        This method returns the saved timestamps & values as a dictionary of "timestamp: value" pairs.
        """
        
        return dict(zip(self.timestamps, self.values))
    
    def set_per_share_data(self, *args, **kwargs):
        """
        This method sets the per share values for the current metric.
        """
        
        if 'x_total' in kwargs and 'x_per_share' in kwargs:
            if len(kwargs['x_total']) != len(self.values) or len(kwargs['x_total']) != len(kwargs['x_per_share']):
                raise ValueError('Invalid length(s) of values.')
            else:
                _total_shares = np.array(kwargs['x_total']) / kwargs['x_per_share']
                self.per_share_values = list(np.array(self.values) / _total_shares)
                
    def get_per_share_data(self):
        """
        This method returns the saved timestamps & per share values as a dictionary of
        "<timestamp>: <per share value>"
        """
        
        return dict(zip(self.timestamps, self.per_share_values))
        
        
# testing
aapl_ebit = FinancialMetric(name = 'ebit', 
                            timestamps = aapl_financials_history['financials']['annuals']['Fiscal Year'],
                            start_year = start_year_financials_history,
                            values = aapl_financials_history['financials']['annuals']['income_statement']['EBIT'])
aapl_ebit.get_raw_data()

### <font color=red>TODO - revamp the current logic for calculating per share data</font>

- Use the "sharings outstanding (average diluated)" data from Guru (income statement) instead

In [ ]:
aapl_revenue = FinancialMetric(name='revenue',
                               timestamps=aapl_financials_history['financials']['annuals']['Fiscal Year'],
                               start_year=start_year_financials_history,
                               values=aapl_financials_history['financials']['annuals']['income_statement']['Revenue'])
aapl_revenue_per_share = FinancialMetric(name='revenue per share',
                                         timestamps=aapl_financials_history['financials']['annuals']['Fiscal Year'],
                                         start_year=start_year_financials_history,
                                         values=aapl_financials_history['financials']['annuals']['per_share_data_array']['Revenue per Share'])

In [ ]:
aapl_ebit.set_per_share_data(x_total=aapl_revenue.values, x_per_share=aapl_revenue_per_share.values)
aapl_ebit.get_per_share_data()

## Helper function to calculate the average Price-to-X ratio

In [ ]:
def calc_avg_price_ratio(price_data, per_share_metric_data):
    """
    This function calculates and returns the average price-to-metric ratio.
    
    Input:
        - "price_data" - dictionary of "<timestamp>: <price>"
        - "per_share_metric_data" - dictionary of "<timestamp>: <per share metric value>", 
                                    with timestamps at the year level
    """
    
    # convert the keys of the per share metric dictionary from timestamps to years
    _years = [timestamp.year for timestamp in per_share_metric_data.keys()]
    _metric_values_by_year = dict(zip(_years, list(per_share_metric_data.values())))
    
    # get new dictionaries for timestamps where both price and metric value are available
    _merged_data = {}
    for key in price_data:
        _metric_value = _metric_values_by_year.get(key.year)
        if _metric_value:
            _merged_data[key] = (price_data[key], _metric_value, 
                                 price_data[key]/_metric_value)

    # put all price-to-metric ratios into a list
    _ratios = [_merged_data[key][-1] for key in _merged_data]
    
    return sum(_ratios) / len(_ratios)
    

# testing
calc_avg_price_ratio(aapl_quote_history, aapl_ebit.get_per_share_data())

In [ ]:
def get_normal_price(per_share_metric_data, avg_price_to_x_ratio):
    """
    This function calculates and returns the "normal price" with respect to 
    a specific income/cash flow metric X:
    
        "normal price" = "historical price" x "historical average Price-to-X ratio"

    Inputs:
        - "per_share_metric_data": dictionary of "<timestamp>: <per share metric value>"
        - "avg_price_to_x_ratio": a numerical value
        
    Output:
        - "normal price": see above
    """
    
    return {timestamp: max(0, per_share_metric_data[timestamp] * avg_price_to_x_ratio) \
            for timestamp in per_share_metric_data}


# testing
aapl_normal_price_per_ebit = get_normal_price(aapl_ebit.get_per_share_data(), 
                                              calc_avg_price_ratio(aapl_quote_history, aapl_ebit.get_per_share_data()))
aapl_normal_price_per_ebit

## Chart price vs normal price (per metric X) 

In [ ]:
from bokeh.models import DatetimeTickFormatter, NumeralTickFormatter
from bokeh.plotting import figure, show
from bokeh.io import output_notebook

output_notebook()

In [ ]:
def plot(quote_history_data, normal_price_data):
    """
    This helper function
    """
    
    # create new plot
    p = figure(title = "Price Correlated with Fundamentals",
               x_axis_type = "datetime",
               x_axis_label= "Time",
               y_axis_label= "Price")

    # add a line for historical prices by month
    p.line(list(quote_history_data.keys()), 
           list(quote_history_data.values()), 
           legend_label='Price', 
           color='black',
           line_width=2)

    # add a line for "normal prices" by year
    p.line(list(normal_price_data.keys()),
           list(normal_price_data.values()),
           legend_label = 'Normal Price',
           line_width = 2)

    show(p)
    

# testing
plot(aapl_quote_history, aapl_normal_price_per_ebit)

## Experiment

In [ ]:
# parameters - choose a company
symbol = 'CAT'

# get financial history
financials_history = get_guru_data(symbol, 'financials')

In [ ]:
# parameters - experiment

# set years interval for analysis
analysis_interval_years = 20

# financials parameters
section_name = 'income_statement'
metric_name = 'EBITDA'

In [ ]:
# get the earlist fiscal year accordingly
start_year = datetime.utcnow().year - analysis_interval_years
start_date_quote_history = datetime(start_year, 1, 1)
start_year_financials_history = start_year - 1

# get quote history
quote_history = get_quote_history(symbol, start=start_date_quote_history.strftime('%Y-%m-%d'),
                                  end=datetime.utcnow().strftime('%Y-%m-%d'))

# get revenue and revenue per share
revenue = FinancialMetric(name='revenue',
                          timestamps=financials_history['financials']['annuals']['Fiscal Year'],
                          start_year=start_year_financials_history,
                          values=financials_history['financials']['annuals']['income_statement']['Revenue'])
revenue_per_share = FinancialMetric(name='revenue per share',
                                    timestamps=financials_history['financials']['annuals']['Fiscal Year'],
                                    start_year=start_year_financials_history,
                                    values=financials_history['financials']['annuals']['per_share_data_array']['Revenue per Share'])

# get per share metric data
metric_X = FinancialMetric(name = metric_name, 
                           timestamps = financials_history['financials']['annuals']['Fiscal Year'],
                           start_year = start_year_financials_history,
                           values = financials_history['financials']['annuals'][section_name][metric_name])

metric_X.set_per_share_data(x_total=revenue.values, x_per_share=revenue_per_share.values)

# get normal prices for metric X
normal_price_per_X = get_normal_price(metric_X.get_per_share_data(), 
                                      calc_avg_price_ratio(quote_history, metric_X.get_per_share_data()))

# plot
plot(quote_history, normal_price_per_X)

# <font color=blue>Moderate data issues with per share financials from Guru</font>

- It seems like the per share financials from Guru is not always properly split adjusted
- For example, there was a 2-to-1 split in 2004 for "CAT"
- And the per share "EBIDTA" value for 2004 from the Guru API (see below) are clearly not properly adjusted
- The # of outstanding shares data for 2004 is also obviously wrong
- Tested a few more companies:
    - Checked "AAPL" and "EPD"
        - Both companies had splits
        - <font color=blue>**Good news**</font> - It seems both the # of outstanding shares data and the per share financials data were correct for years close to the splits
- Some minor differences were also observed when comparing for example "EBIT/share" from FastGraphs and Guru
    - Guru's total EBIT (using Apple as an example) seemed to be consistent with that of Yahoo Finance
    - Guru's shares outstanding (average diluted) also seemed to be consistent with that of Yahoo Finance
    - So despite some minor differences noticed when compared to FastGraphs, Guru's data should be mostly good
    
<font color=blue>**Conclusions**</font>
- Guru's financials data seems to be mostly alright - nothing much to worry about

In [ ]:
financials_history['financials']['annuals']['income_statement'].keys()

In [ ]:
# get oustanding shares by year
dict(zip(financials_history['financials']['annuals']['Fiscal Year'], financials_history['financials']['annuals']['income_statement']['Shares Outstanding (Diluted Average)']))

In [ ]:
# get EBIDTA per share by year
dict(zip(financials_history['financials']['annuals']['Fiscal Year'], financials_history['financials']['annuals']['per_share_data_array']['EBITDA per Share']))

# <font color=red>Severe data issues with quote data and next steps</font> 

- The historical quotes returned by the yfinance API (by RAN AROUSSI) are not correct for dates in earlier years
    - For example, see the tests below for 'CAT'. I also observed issues for at least 'EPD'.
- **Next steps**: test and validate other options to get historical quotes, such as the 'yahoo_fin' library
    - See a guide here for the 'yahoo_fin' library - https://algotrading101.com/learn/yahoo-finance-api-guide/

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', None)

In [ ]:
_stock = yf.Ticker('CAT')
_stock.history(interval='1mo', 
               start=start_date_quote_history.strftime('%Y-%m-%d'),
               end=datetime.utcnow().strftime('%Y-%m-%d'))['Close']

In [ ]:
_stock.splits

In [ ]:
_stock.history(interval='1mo', 
               start=start_date_quote_history.strftime('%Y-%m-%d'),
               end=datetime.utcnow().strftime('%Y-%m-%d'))